In [1]:
import pymysql
import datetime
import pandas as pd
import numpy as np
import joblib
import os
import logging
import time
import json
import warnings
warnings.filterwarnings('ignore')

In [2]:
def getPlanData(conn, begin, end):
    originSql = '''
        SELECT
            a.image_id AS 'image_id',
            a.image_name AS 'image_name',
            a.media_id,
            a.label_ids,
            c.launch_time AS 'image_launch_time',
            c.image_source_total_num AS 'image_source_total_num',
            ifnull( a.image_run_date_amount, 0 ) AS 'image_run_date_amount',
            ifnull( a.image_create_role_pay_num, 0 ) AS 'image_create_role_pay_num',
            ifnull( a.image_create_role_num, 0 ) AS 'image_create_role_num',
            ifnull( a.image_create_role_pay_sum, 0 ) AS 'image_create_role_pay_sum',
            ifnull( a.image_source_num, 0 ) AS 'image_source_num',
            (
            CASE

                    WHEN ifnull( a.image_create_role_num, 0 )= 0 THEN
                    0 ELSE IFNULL( a.image_create_role_pay_num, 0 ) / ifnull( a.image_create_role_num, 0 ) 
                END 
                ) AS 'image_create_role_pay_rate',
                (
                CASE

                        WHEN ifnull( a.image_create_role_num, 0 )= 0 THEN
                        0 ELSE IFNULL( a.image_run_date_amount, 0 ) / ifnull( a.image_create_role_num, 0 ) 
                    END 
                    ) AS 'image_create_role_cost',
                    (
                    CASE

                            WHEN ifnull( a.image_create_role_pay_num, 0 )= 0 THEN
                            0 ELSE IFNULL( a.image_run_date_amount, 0 ) / ifnull( a.image_create_role_pay_num, 0 ) 
                        END 
                        ) AS 'image_create_role_pay_cost',
                        ifnull( b.image_valid_source_num, 0 ) AS 'image_valid_source_num',
                        (
                        CASE

                                WHEN ifnull( a.image_source_num, 0 )= 0 THEN
                                0 ELSE IFNULL( b.image_valid_source_num, 0 ) / ifnull( a.image_source_num, 0 ) 
                            END 
                            ) AS 'image_valid_source_rate',
                            (
                            CASE

                                    WHEN ifnull( b.image_valid_source_num, 0 )= 0 THEN
                                    0 ELSE IFNULL( a.image_create_role_pay_sum, 0 ) / ifnull( b.image_valid_source_num, 0 ) 
                                END 
                                ) AS 'image_pay_sum_ability',
                                (
                                CASE

                                        WHEN ifnull( b.image_valid_source_num, 0 )= 0 THEN
                                        0 ELSE IFNULL( a.image_create_role_pay_num, 0 ) / ifnull( b.image_valid_source_num, 0 ) 
                                    END 
                                    ) AS 'image_pay_num_ability',
                                    (
                                    CASE

                                            WHEN ifnull( a.image_run_date_amount, 0 )= 0 THEN
                                            0 ELSE IFNULL( a.image_create_role_pay_sum, 0 ) / ifnull( a.image_run_date_amount, 0 ) 
                                        END 
                                        ) AS 'image_create_role_roi' 
                                    FROM
                                        (
                                        SELECT
                                            b.image_id,
                                            c.image_name,
                                            c.label_ids,
                                            a.media_id,
                                            sum( a.amount ) AS 'image_run_date_amount',
                                            sum( pay_role_user_num ) AS 'image_create_role_pay_num',
                                            sum( create_role_num ) AS 'image_create_role_num',
                                            sum( new_role_money ) AS 'image_create_role_pay_sum',
                                            count( DISTINCT b.plan_id ) AS 'image_source_num' 
                                        FROM
                                            db_stdata.st_lauch_report a
                                            INNER JOIN db_data_ptom.ptom_plan b ON a.game_id = b.game_id 
                                            AND a.source_id = b.source_id 
                                            AND a.channel_id = b.chl_user_id
                                            LEFT JOIN db_data_ptom.ptom_image_info c ON b.image_id = c.image_id 
                                        WHERE
                                            a.tdate_type = 'day'
                                          AND a.media_id in (10,16)	
                                            AND a.tdate >= '{begin}' 
                                            AND a.tdate <= '{end}' AND a.amount > 100 
                                            AND b.image_id IS NOT NULL 
                                            AND b.image_id <> '' 
                                            AND a.game_id IN ( SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1056 AND dev_game_id IS NOT NULL ) 
                                        GROUP BY
                                            b.image_id,
                                            c.image_name,
                                            c.label_ids,
                                            a.media_id	
                                        ) a
                                        LEFT JOIN (
                                        SELECT
                                            c.image_id,
                                            c.media_id,
                                            sum( c.image_valid_source_num ) AS 'image_valid_source_num' 
                                        FROM
                                            (
                                            SELECT
                                                b.image_id,
                                                a.media_id,
                                                count( DISTINCT b.plan_id ) AS 'image_valid_source_num' 
                                            FROM
                                                db_stdata.st_lauch_report a
                                                INNER JOIN db_data_ptom.ptom_plan b ON a.game_id = b.game_id 
                                                AND a.source_id = b.source_id 
                                                AND a.channel_id = b.chl_user_id 
                                            WHERE
                                                a.tdate_type = 'day' 
                                                AND a.tdate >= '{begin}' 
                                                AND a.tdate <= '{end}' AND a.amount > 100
                                                AND a.media_id in (10,16)
                                                AND b.image_id IS NOT NULL 
                                                AND b.image_id <> '' 
                                                AND a.game_id IN ( SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1056 AND dev_game_id IS NOT NULL ) 
                                                AND a.platform = 1 
                                            GROUP BY
                                                b.image_id,
                                                a.media_id
                                            HAVING
                                                sum( a.amount ) / sum( a.pay_role_user_num )< 3500 
                                                AND sum( a.pay_role_user_num )> 0 
                                            UNION ALL
                                            SELECT
                                                b.image_id,
                                                a.media_id,
                                                count( DISTINCT b.plan_id ) AS 'image_valid_source_num' 
                                            FROM
                                                db_stdata.st_lauch_report a
                                                INNER JOIN db_data_ptom.ptom_plan b ON a.game_id = b.game_id 
                                                AND a.source_id = b.source_id 
                                                AND a.channel_id = b.chl_user_id 
                                            WHERE
                                                a.tdate_type = 'day' 
                                                AND a.tdate >= '{begin}' 
                                                AND a.tdate <= '{end}' AND a.amount > 100
                                              AND a.media_id in (10,16)	
                                                AND b.image_id IS NOT NULL 
                                                AND b.image_id <> '' 
                                                AND a.game_id IN ( SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1056 AND dev_game_id IS NOT NULL ) 
                                                AND a.platform = 2 
                                            GROUP BY
                                                b.image_id,
                                                a.media_id	
                                            HAVING
                                                SUM( a.amount ) / SUM( a.pay_role_user_num ) < 5000 AND SUM( a.pay_role_user_num ) > 0 
                                            ) c 
                                        GROUP BY
                                            c.image_id,
                                            c.media_id	
                                        ) b ON a.image_id = b.image_id and a.media_id = b.media_id
                                        LEFT JOIN (
                                        SELECT
                                            b.image_id,
                                            min( b.launch_time ) AS launch_time,
                                            count( DISTINCT b.plan_id ) AS image_source_total_num 
                                        FROM
                                            db_stdata.st_lauch_report a
                                            INNER JOIN db_data_ptom.ptom_plan b ON a.game_id = b.game_id 
                                            AND a.source_id = b.source_id 
                                            AND a.channel_id = b.chl_user_id 
                                        WHERE
                                            a.tdate_type = 'day' 
                                            AND a.amount > 100 
                                            AND b.image_id IS NOT NULL 
                                            AND b.image_id <> '' 
                                            AND a.game_id IN ( SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1056 AND dev_game_id IS NOT NULL ) 
                                        GROUP BY
                                        b.image_id 
            ) c ON a.image_id = c.image_id
    '''
    finalSql = originSql.format(begin=begin, end=end)
    result = pd.read_sql(finalSql, conn)
    return result

In [3]:
# 链接数据库，并创建游标
conn1 = pymysql.connect(host='192.168.0.79', port=3306, user='readonly',
                        passwd='Kc&r+z1ql9x8za4kzfk761weq8ozGv3ZpI;RMe,#+s%c>t', db='db_data')
# cur1 = conn1.cursor(cursor=pymysql.cursors.DictCursor)

end = datetime.datetime.now().strftime('%Y-%m-%d')
begin = (datetime.datetime.now() - pd.Timedelta(days=3)).strftime('%Y-%m-%d')
# 获取计划数据
planDataList = getPlanData(conn1, begin, end)

In [4]:
planDataList.head()

,image_id,image_name,media_id,label_ids,image_launch_time,image_source_total_num,image_run_date_amount,image_create_role_pay_num,image_create_role_num,image_create_role_pay_sum,image_source_num,image_create_role_pay_rate,image_create_role_cost,image_create_role_pay_cost,image_valid_source_num,image_valid_source_rate,image_pay_sum_ability,image_pay_num_ability,image_create_role_roi
0,16195,Mxz096,10,None,2021-04-21,1,446.63,0.0,6.0,0.0,1,0.0000,74.438333,0.00,0.0,0.0,0.0,0.0,0.000000
1,19636,SSR710-横版（混剪)-640x360—30S 100M内,16,19,2019-11-15,863,435.99,0.0,1.0,0.0,1,0.0000,435.990000,0.00,0.0,0.0,0.0,0.0,0.000000
2,22316,SSR1291-横版（混剪）,16,"-1,19",2020-06-16,227,4984.38,0.0,40.0,0.0,2,0.0000,124.609500,0.00,0.0,0.0,0.0,0.0,0.000000
3,22326,SSR1298-横版（混剪）,16,22,2020-06-16,227,231.61,0.0,1.0,0.0,1,0.0000,231.610000,0.00,0.0,0.0,0.0,0.0,0.000000
4,24402,SSR1588-横版（端游混剪）,16,22,2020-08-17,560,9428.66,1.0,192.0,256.0,13,0.0052,49.107604,9428.66,0.0,0.0,0.0,0.0,0.027151
